In [17]:
from sqlalchemy import create_engine
import pandas as pd
import yaml
from urllib.parse import quote_plus

In [18]:
def import_credentials(file_path):
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)
    return data

In [21]:
class RDSDatabaseConnector:
    def __init__(self, data):
        self.data = data
        self.engine= None

    def initialize_engine(self):
        HOST = quote_plus(self.data['RDS_HOST'])
        USER = quote_plus(self.data['RDS_USER'])
        PASSWORD = self.data['RDS_PASSWORD']
        DATABASE = self.data['RDS_DATABASE']
        PORT = self.data['RDS_PORT']
                
        # Create the connection URL for SQLAlchemy
        connection_url = f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
        
        # Create SQLAlchemy engine
        self.engine = create_engine(connection_url)
        print("Engine initialized successfully.")
    
    def extract_data(self, table_name="loan_payments"):
        if self.engine is None:
            raise ValueError("Engine not initialized. Please call 'initialize_engine' first.")
        
        # Query the table and load data into a Pandas DataFrame
        query = f"SELECT * FROM {table_name};"
        df = pd.read_sql(query, self.engine)
        return df

    def save_data_to_csv(self, data, file_name="loan_payments_data.csv"):
        data.to_csv(file_name, index=False)
        print(f"Data saved to {file_name}.")

In [22]:
data = import_credentials('C:/Users/paven/aicore/eda/project/exploratory-data-analysis---customer-loans-in-finance971/credentials.yaml')
db_connector = RDSDatabaseConnector(data)
db_connector.initialize_engine()
data_extract = db_connector.extract_data()
db_connector.save_data_to_csv(data_extract, 'loan_payments_data.csv')

Engine initialized successfully.
Data saved to loan_payments_data.csv.


In [23]:
def load_data_from_csv(file_path):
    try:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        print(f"Data successfully loaded from {file_path}.")
        return df
    except FileNotFoundError:
        print(f"Error: The file at {file_path} does not exist.")
        return None
    except pd.errors.EmptyDataError:
        print(f"Error: The file at {file_path} is empty.")
        return None
    except pd.errors.ParserError:
        print(f"Error: There was an issue parsing the file at {file_path}.")
        return None


In [27]:
data = load_data_from_csv("C:/Users/paven/aicore/eda/project/exploratory-data-analysis---customer-loans-in-finance971/loan_payments_data.csv")
if data is not None:
    print(data.head())

Data successfully loaded from C:/Users/paven/aicore/eda/project/exploratory-data-analysis---customer-loans-in-finance971/loan_payments_data.csv.
         id  member_id  loan_amount  funded_amount  funded_amount_inv  \
0  38676116   41461848         8000         8000.0             8000.0   
1  38656203   41440010        13200        13200.0            13200.0   
2  38656154   41439961        16000        16000.0            16000.0   
3  38656128   41439934        15000        15000.0            15000.0   
4  38656121   41439927        15000        15000.0            15000.0   

        term  int_rate  instalment grade sub_grade  ... recoveries  \
0  36 months      7.49      248.82     A        A4  ...        0.0   
1  36 months      6.99      407.52     A        A3  ...        0.0   
2  36 months      7.49      497.63     A        A4  ...        0.0   
3  36 months     14.31      514.93     C        C4  ...        0.0   
4  36 months      6.03      456.54     A        A1  ...        0.0